In [1]:
#importamos paquetes
import sys
sys.path.append('Src/')
from data_modificado import * #hay funciones que estan cambiadas en este script para adaptralas a nuestro dataset
from train_2 import * #este hubo que modificar una linea tambien
from transfer_learning import * #hubo que modificart lo mismo que en train_2
from test_functions import *
from layers import *
from utils import *
from loss import *
from metric import *
from results import *
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
import itertools as it

#funciones
def read_df(
              metadata_names=['age','Temperature','Precipitation3Days'],
              random_state=42,
              otu_filename='../Datasets/otu_table_all_80.csv',
              metadata_filename='../Datasets/metadata_table_all_80.csv'):
    otu = pd.read_csv(otu_filename, index_col=0, header=None).T
    #print(otu.head())
    otu = otu.set_index('otuids')
    otu = otu.astype('int32')
    metadata = pd.read_csv(metadata_filename)
    #print(metadata.head())
    metadata = metadata.set_index('X.SampleID')
    metadata.head()
    domain = metadata[metadata_names]
    #if 'INBREDS' in metadata_names:
    #    domain = pd.concat([domain, pd.get_dummies(domain['INBREDS'], prefix='INBREDS')], axis=1)
    #    domain = domain.drop(['INBREDS'], axis=1)
    #elif 'Maize_Line' in metadata_names:
    #    domain = pd.concat([domain, pd.get_dummies(domain['Maize_Line'], prefix='Maize_Line')], axis=1)
    #    domain = domain.drop(['Maize_Line'], axis=1) 
    df = pd.concat([otu, domain], axis=1, sort=True, join='outer')
    #print(df.head())
    #data_microbioma = df[otu.columns].to_numpy(dtype=np.float32)
    #data_domain = df[domain.columns].to_numpy(dtype=np.float32)
    df_microbioma = df[otu.columns]
    df_domain = df[domain.columns]
    df_domain.head()
    df_microbioma_train, df_microbioma_no_train, df_domain_train, df_domain_no_train = \
        train_test_split(df_microbioma, df_domain, test_size=0.1, random_state=random_state)
    # Transfer learning subset
    df_microbioma_test, df_microbioma_transfer_learning, df_domain_test, df_domain_transfer_learning = \
        train_test_split(df_microbioma_no_train, df_domain_no_train, test_size=0.1, random_state=random_state)
    df_microbioma_transfer_learning_train, df_microbioma_transfer_learning_test, df_domain_transfer_learning_train, df_domain_transfer_learning_test = \
        train_test_split(df_microbioma_transfer_learning, df_domain_transfer_learning, test_size=0.3, random_state=random_state)
    
    return df_microbioma_train, df_microbioma_test, df_microbioma_transfer_learning_train, df_microbioma_transfer_learning_test, df_domain_train, df_domain_test, df_domain_transfer_learning_train, df_domain_transfer_learning_test, otu.columns, domain.columns
    #return df_microbioma_train, df_microbioma_test, df_microbioma_transfer_learning_train, df_microbioma_transfer_learning_test, df_domain_train, df_domain_test, df_domain_transfer_learning_train, df_domain_transfer_learning_test, otu.columns, domain.columns


def train_kfold_mod(model_fn, m_train, d_train, z_train, m_test, d_test, z_test,
                batch_size, epochs, train_callbacks):
    all_models = model_fn()
    model, encoder_bioma, encoder_domain, decoder_bioma = all_models
    metrics_prefix = None
    if encoder_bioma is not None and encoder_domain is not None:
        x_train = (m_train, d_train)
        y_train = (m_train, m_train, z_train)
        x_test = (m_test, d_test)
        y_test = (m_test, m_test, z_test)
    elif encoder_bioma is not None:
        x_train = m_train
        y_train = m_train
        x_test = m_test
        y_test = m_test
        metrics_prefix = 'bioma'
    elif encoder_domain is not None:
        x_train = d_train
        y_train = m_train
        x_test = d_test
        y_test = m_test
        metrics_prefix = 'domain'

    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(5000).batch(
        batch_size)
    train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
    val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
    val_dataset = val_dataset.prefetch(tf.data.experimental.AUTOTUNE)
    r = model.fit(train_dataset,
                  epochs=epochs,
                  validation_data=val_dataset,
                  callbacks=train_callbacks,
                  verbose=0)
    if metrics_prefix is not None:
        old_keys = r.history
        r.history = {}
        for k, v in old_keys.items():
            if k == 'loss' or k == 'val_loss':
                new_key = k
            elif k.startswith('val_'):
                new_key = 'val_{}_{}'.format(metrics_prefix, k[4:])
            else:
                new_key = '{}_{}'.format(metrics_prefix, k)
            r.history[new_key] = v
    del val_dataset
    del train_dataset
    del x_train
    del y_train
    del x_test
    del y_test
    return r, all_models

def train_2(model_fn,
          data_microbioma,
          data_domain,
          latent_space=10,
          folds=5,
          epochs=20,
          batch_size=128,
          learning_rate_scheduler=ExpDecayScheluder(),
          random_seed=347,
          verbose=0):
    data_zeros_latent = np.zeros((data_microbioma.shape[0], latent_space), dtype=data_microbioma.dtype)
    results = []
    models = []
    train_callbacks = [
        callbacks.EarlyStopping(monitor='val_loss', patience=epochs + 1, restore_best_weights=True)]
    if verbose >= 0:
        train_callbacks += [TqdmCallback(verbose=verbose)]
    if learning_rate_scheduler is not None:
        train_callbacks += [learning_rate_scheduler.make()]

    if folds <= 1:
        m_train, m_test = data_microbioma, data_microbioma
        d_train, d_test = data_domain, data_domain
        z_train, z_test = data_zeros_latent, data_zeros_latent
        tf.random.set_seed(random_seed)
        r, m = train_kfold(model_fn, m_train, d_train, z_train, m_test, d_test, z_test,
                           batch_size, epochs, train_callbacks)
        results.append(r)
        models.append(m)

    else: #EL PROBLEMA ESTA AQUI, QUE HACE FALTA UN 
        kf = KFold(n_splits=folds, random_state=random_seed, shuffle=True)
        tf.random.set_seed(random_seed)

        for train_index, test_index in kf.split(data_microbioma):
            m_train, m_test = data_microbioma[train_index], data_microbioma[test_index]
            #print(m_train)
            #d_train, d_test = data_domain[train_index], data_domain[test_index]
            if data_domain is None:
                d_train, d_test = None, None
            else:
                d_train, d_test = data_domain[train_index], data_domain[test_index]
            #print(d_train)
            #Esto de hacer el if else ha funcionado, pero no se si hace lo que debe bien
            z_train, z_test = data_zeros_latent[train_index], data_zeros_latent[test_index]
            r, m = train_kfold_mod(model_fn, m_train, d_train, z_train, m_test, d_test, z_test,
                               batch_size, epochs, train_callbacks)
            results.append(r)
            models.append(m)
    return results, models

def perform_experiment_2_mod(cv_folds, epochs, batch_size, learning_rate, optimizer,
                       learning_rate_scheduler, input_transform, output_transform,
                       reconstruction_loss, latent_space, layers,
                       activation, activation_latent,
                       data_microbioma_train, data_domain_train,
                       show_results=True, device='/CPU:0'): #Show results cambiado de False  aTrue
    if input_transform is not None:
        input_transform = input_transform()
    #----------    
    if output_transform is not None:
        output_transform = output_transform()
    #----------      
    if reconstruction_loss.__class__.__name__ == 'MakeLoss':
        reconstruction_loss = reconstruction_loss.make()
    else:
        reconstruction_loss = reconstruction_loss()
    domain_layers = [l // 16 for l in layers] ####que es esto???? Esto es para las capas del domain
    #print(domain_layers)
    bioma_autoencoder = " -> ".join(["b"] +
                                    [str(l) for l in layers] +
                                    [str(latent_space)] +
                                    [str(l) for l in reversed(layers)] +
                                    ["b"])
    #---------- 
    #esto solo se utiliza para el texto, es irrelevante para nuestro error
    if data_domain_train is not None:
        domain_autoencoder = " -> ".join(["d"] +
                                     [str(l) for l in domain_layers] +
                                     [str(latent_space)] +
                                     [str(l) for l in reversed(layers)] +
                                     ["b"])
        
    else: 
        domain_autoencoder = " "
    #---------- 
    #donde se usa domain autoencoder?
    in_transform_name = input_transform.__class__.__name__ if input_transform else "none"
    out_transform_name = output_transform.__class__.__name__ if output_transform else "none"
    lr_scheduler_text = learning_rate_scheduler[
        1] if learning_rate_scheduler is not None else "none"
    lr_text = learning_rate if learning_rate_scheduler is not None else "constant = {}".format(
        learning_rate)
    learning_rate_scheduler = learning_rate_scheduler[
        0] if learning_rate_scheduler is not None else None
    optimizer = optimizer(learning_rate=learning_rate)
    #---------- 
    experiment_parameters = [
        ("Input transform", in_transform_name),
        ("Output transform", out_transform_name),
        ("Reconstruction Loss", reconstruction_loss.__class__.__name__),
        ("Latent Space", latent_space),
        ("Bioma Autoencoder", bioma_autoencoder),
        ("Domain Autoencoder", domain_autoencoder),
        ("Activation Encoder", activation),
        ("Activation Decoder", activation),
        ("Activation Latent", activation_latent),
        ("CV folds", cv_folds),
        ("Epochs", epochs),
        ("Batch Size", batch_size),
        ("Learning Rate Scheduler", lr_scheduler_text),
        ("Learning Rate", lr_text),
        ("Optimizer", optimizer.__class__.__name__),
    ]
    #----------  
    if show_results:
        md_text = ""
        md_text += "| Parameter             | Value         |\n"
        md_text += "|:----------------------|:--------------|\n"
        for n, v in experiment_parameters:
            md_text += "| {} | {} |\n".format(n, v)

        display(Markdown(md_text))
    #------------
    def create_model(print_data=False):
        bioma_shape=data_microbioma_train.shape[1]
        
        if data_domain_train is not None:
            domain_shape=data_domain_train.shape[1]
            #print("data_domain_train!=None")
        else:
            domain_shape=None
            #print("data_domain_train==None")
        models = autoencoder(bioma_shape=bioma_shape,
                             #bioma_shape=717,
                             domain_shape=domain_shape,
                             output_shape=bioma_shape,
                             #output_shape=717,
                             latent_space=latent_space,
                             bioma_layers=layers, #Esto es lo de [512,316]
                             domain_layers=domain_layers, #Esto son cada una de las layers divididas por 16
                             input_transform=input_transform,
                             output_transform=output_transform,
                             activation_function_encoder=activation,
                             activation_function_decoder=activation,
                             activation_function_latent=activation_latent)
        #Entiendo analizando lo demas que aqui NO esta el error
        #la funcion autoencoder esta en model.py (es la unica funcion en ese script)
        
        model, encoder_bioma, encoder_domain, decoder_bioma = models

        if print_data:
            plot_models(model, encoder_bioma, encoder_domain, decoder_bioma)
        compile_train(model,
                      encoder_bioma=encoder_bioma,
                      encoder_domain=encoder_domain,
                      reconstruction_error=reconstruction_loss,
                      encoded_comparison_error=losses.MeanAbsoluteError(),
                      metrics=get_experiment_metrics(input_transform, output_transform),
                      optimizer=optimizer)
        
        #print("He acabado create_model :)")
        return model, encoder_bioma, encoder_domain, decoder_bioma
    #-----------
    create_model(print_data=False)
    #-----------
    #Esta en esta seccion el problema, en train_2
    #print(data_domain_train)
    #print(latent_space)
    with tf.device(device):
        results, models = train_2(create_model,
                                data_microbioma_train,
                                data_domain_train,
                                latent_space=latent_space,
                                folds=cv_folds,
                                epochs=epochs,
                                batch_size=batch_size,
                                learning_rate_scheduler=learning_rate_scheduler,
                                verbose=-1)
    #----------
    validation_results = print_results(results, show_results=show_results)
    if show_results:
        display(Markdown("*************"))

    return experiment_parameters + validation_results, models, results

__params:__
 - "activat_func":["softmax","sigmoid","relu","tanh"]
 - "activ_ouput":["softmax","sigmoid","relu","tanh"]
 - "learning_rate":[0.01,0.001]

In [2]:
#Cargamos los datos
#nombres_metadatos = ["KCAL","PROT","TFAT","CARB","MOIS","ALC","CAFF","THEO","SUGR","FIBE","CALC","IRON","MAGN","PHOS","POTA","SODI","ZINC","COPP","SELE","VC","VB1","VB2","NIAC","VB6","FOLA","VB12","VARA","RET","BCAR","ACAR","CRYP","LYCO","LZ","ATOC","VK","CHOLE","SFAT","MFAT","PFAT","VITD","CHOLN"]
nombres_metadatos = ['F_TOTAL', 'V_TOTAL', 'V_REDOR_TOTAL', 'V_STARCHY_TOTAL', 'V_LEGUMES', 'G_TOTAL', 'PF_TOTAL', 'PF_MEAT', 'PF_CUREDMEAT', 'PF_ORGAN', 'PF_POULT',\
					'PF_SEAFD_HI', 'PF_SEAFD_LOW', 'PF_EGGS', 'PF_SOY', 'PF_NUTSDS', 'D_TOTAL', 'D_YOGURT', 'OILS', 'SOLID_FATS', 'ADD_SUGARS', 'A_DRINKS']
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = read_df(metadata_names=nombres_metadatos,otu_filename='resultados_ana/datos_otus_metadatos_especies/otu_table_especies_80.csv',metadata_filename='datos-remoto/comidas/metadatos_comidas.csv')

data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

#Preparamos las combinaciones pertinentes (5 mejores)
combinations = [[100,64,0.001,optimizers.Adam,15,[512,256],"tanh","tanh"],\
                [100,64,0.001,optimizers.Adam,15,[512,256,128],"tanh","relu"],\
                [100,96,0.001,optimizers.Adam,15,[512,256],"tanh","tanh"],\
                [100,96,0.01,optimizers.Adam,15,[512,256],"relu","tanh"],\
                [100,64,0.01,optimizers.Adam,10,[512,256],"relu","tanh"]]

### 0.001, tanh, tanh

In [3]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation=None)(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4805366098880768 | 0.4805366098880768 | 0.4805366098880768 |
| pearson_corr | 0.625096321105957 | 0.625096321105957 | 0.625096321105957 |
| jensen_shannon_divergence | 1.2776437997817993 | 1.2776437997817993 | 1.2776437997817993 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4775315225124359 | 0.4775315225124359 | 0.4775315225124359 |
| pearson_corr | 0.5913827419281006 | 0.5913827419281006 | 0.5913827419281006 |
| jensen_shannon_divergence | 1.4342634677886963 | 1.4342634677886963 | 1.4342634677886963 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5001455545425415 | 0.5001455545425415 | 0.5001455545425415 |
| pearson_corr | 0.561398446559906 | 0.561398446559906 | 0.561398446559906 |
| jensen_shannon_divergence | 1.5791258811950684 | 1.5791258811950684 | 1.5791258811950684 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4478285014629364 | 0.4478285014629364 | 0.4478285014629364 |
| pearson_corr | 0.638617753982544 | 0.638617753982544 | 0.638617753982544 |
| jensen_shannon_divergence | 1.333242416381836 | 1.333242416381836 | 1.333242416381836 |


----------------------------



### 0.001, tanh, softmax

In [4]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6001875996589661 | 0.6001875996589661 | 0.6001875996589661 |
| pearson_corr | 0.5634642839431763 | 0.5634642839431763 | 0.5634642839431763 |
| jensen_shannon_divergence | 1.6034209728240967 | 1.6034209728240967 | 1.6034209728240967 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6327547430992126 | 0.6327547430992126 | 0.6327547430992126 |
| pearson_corr | 0.490217000246048 | 0.490217000246048 | 0.490217000246048 |
| jensen_shannon_divergence | 1.8718141317367554 | 1.8718141317367554 | 1.8718141317367554 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5906188488006592 | 0.5906188488006592 | 0.5906188488006592 |
| pearson_corr | 0.4771423041820526 | 0.4771423041820526 | 0.4771423041820526 |
| jensen_shannon_divergence | 1.763100504875183 | 1.763100504875183 | 1.763100504875183 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4927397072315216 | 0.4927397072315216 | 0.4927397072315216 |
| pearson_corr | 0.6220669746398926 | 0.6220669746398926 | 0.6220669746398926 |
| jensen_shannon_divergence | 1.363389253616333 | 1.363389253616333 | 1.363389253616333 |


----------------------------



### 0.001, tanh, sigmoid

In [5]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5307955741882324 | 0.5307955741882324 | 0.5307955741882324 |
| pearson_corr | 0.5753774642944336 | 0.5753774642944336 | 0.5753774642944336 |
| jensen_shannon_divergence | 1.3179583549499512 | 1.3179583549499512 | 1.3179583549499512 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4673141837120056 | 0.4673141837120056 | 0.4673141837120056 |
| pearson_corr | 0.6193957328796387 | 0.6193957328796387 | 0.6193957328796387 |
| jensen_shannon_divergence | 1.2525559663772583 | 1.2525559663772583 | 1.2525559663772583 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.562671959400177 | 0.562671959400177 | 0.562671959400177 |
| pearson_corr | 0.5628306865692139 | 0.5628306865692139 | 0.5628306865692139 |
| jensen_shannon_divergence | 1.5894299745559692 | 1.5894299745559692 | 1.5894299745559692 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47529488801956177 | 0.47529488801956177 | 0.47529488801956177 |
| pearson_corr | 0.6253515481948853 | 0.6253515481948853 | 0.6253515481948853 |
| jensen_shannon_divergence | 1.3152778148651123 | 1.3152778148651123 | 1.3152778148651123 |


----------------------------



### 0.001, tanh, relu

In [6]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.538262665271759 | 0.538262665271759 | 0.538262665271759 |
| pearson_corr | 0.5426293015480042 | 0.5426293015480042 | 0.5426293015480042 |
| jensen_shannon_divergence | 1.3882941007614136 | 1.3882941007614136 | 1.3882941007614136 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4791297912597656 | 0.4791297912597656 | 0.4791297912597656 |
| pearson_corr | 0.5950087904930115 | 0.5950087904930115 | 0.5950087904930115 |
| jensen_shannon_divergence | 1.398192048072815 | 1.398192048072815 | 1.398192048072815 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.563652753829956 | 0.563652753829956 | 0.563652753829956 |
| pearson_corr | 0.5458134412765503 | 0.5458134412765503 | 0.5458134412765503 |
| jensen_shannon_divergence | 1.6148251295089722 | 1.6148251295089722 | 1.6148251295089722 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5108902454376221 | 0.5108902454376221 | 0.5108902454376221 |
| pearson_corr | 0.6027514934539795 | 0.6027514934539795 | 0.6027514934539795 |
| jensen_shannon_divergence | 1.4575625658035278 | 1.4575625658035278 | 1.4575625658035278 |


----------------------------



### 0.001, softmax, tanh

In [7]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
  
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4675060510635376 | 0.4675060510635376 | 0.4675060510635376 |
| pearson_corr | 0.6249818801879883 | 0.6249818801879883 | 0.6249818801879883 |
| jensen_shannon_divergence | 1.184651494026184 | 1.184651494026184 | 1.184651494026184 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4801386892795563 | 0.4801386892795563 | 0.4801386892795563 |
| pearson_corr | 0.6170819401741028 | 0.6170819401741028 | 0.6170819401741028 |
| jensen_shannon_divergence | 1.2843995094299316 | 1.2843995094299316 | 1.2843995094299316 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5212920308113098 | 0.5212920308113098 | 0.5212920308113098 |
| pearson_corr | 0.5983790755271912 | 0.5983790755271912 | 0.5983790755271912 |
| jensen_shannon_divergence | 1.4273326396942139 | 1.4273326396942139 | 1.4273326396942139 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4947567880153656 | 0.4947567880153656 | 0.4947567880153656 |
| pearson_corr | 0.6046473979949951 | 0.6046473979949951 | 0.6046473979949951 |
| jensen_shannon_divergence | 1.3972156047821045 | 1.3972156047821045 | 1.3972156047821045 |


----------------------------



### 0.001, softmax, softmax

In [8]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')

    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5873197913169861 | 0.5873197913169861 | 0.5873197913169861 |
| pearson_corr | 0.569516658782959 | 0.569516658782959 | 0.569516658782959 |
| jensen_shannon_divergence | 1.5207303762435913 | 1.5207303762435913 | 1.5207303762435913 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6445517539978027 | 0.6445517539978027 | 0.6445517539978027 |
| pearson_corr | 0.4979643225669861 | 0.4979643225669861 | 0.4979643225669861 |
| jensen_shannon_divergence | 1.8684132099151611 | 1.8684132099151611 | 1.8684132099151611 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5904490947723389 | 0.5904490947723389 | 0.5904490947723389 |
| pearson_corr | 0.48134496808052063 | 0.48134496808052063 | 0.48134496808052063 |
| jensen_shannon_divergence | 1.7280449867248535 | 1.7280449867248535 | 1.7280449867248535 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.49683842062950134 | 0.49683842062950134 | 0.49683842062950134 |
| pearson_corr | 0.616696834564209 | 0.616696834564209 | 0.616696834564209 |
| jensen_shannon_divergence | 1.4218618869781494 | 1.4218618869781494 | 1.4218618869781494 |


----------------------------



### 0.001, softmax, sigmoid

In [9]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')

    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5424087047576904 | 0.5424087047576904 | 0.5424087047576904 |
| pearson_corr | 0.5868561267852783 | 0.5868561267852783 | 0.5868561267852783 |
| jensen_shannon_divergence | 1.3533575534820557 | 1.3533575534820557 | 1.3533575534820557 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47496944665908813 | 0.47496944665908813 | 0.47496944665908813 |
| pearson_corr | 0.626322865486145 | 0.626322865486145 | 0.626322865486145 |
| jensen_shannon_divergence | 1.2816084623336792 | 1.2816084623336792 | 1.2816084623336792 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5908095836639404 | 0.5908095836639404 | 0.5908095836639404 |
| pearson_corr | 0.4991908371448517 | 0.4991908371448517 | 0.4991908371448517 |
| jensen_shannon_divergence | 1.6940948963165283 | 1.6940948963165283 | 1.6940948963165283 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.495566725730896 | 0.495566725730896 | 0.495566725730896 |
| pearson_corr | 0.6180101037025452 | 0.6180101037025452 | 0.6180101037025452 |
| jensen_shannon_divergence | 1.41998291015625 | 1.41998291015625 | 1.41998291015625 |


----------------------------



### 0.001, softmax, relu

In [10]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')


    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5407907962799072 | 0.5407907962799072 | 0.5407907962799072 |
| pearson_corr | 0.5861192941665649 | 0.5861192941665649 | 0.5861192941665649 |
| jensen_shannon_divergence | 1.33469557762146 | 1.33469557762146 | 1.33469557762146 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4680374562740326 | 0.4680374562740326 | 0.4680374562740326 |
| pearson_corr | 0.6427944898605347 | 0.6427944898605347 | 0.6427944898605347 |
| jensen_shannon_divergence | 1.2594773769378662 | 1.2594773769378662 | 1.2594773769378662 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5861326456069946 | 0.5861326456069946 | 0.5861326456069946 |
| pearson_corr | 0.5073991417884827 | 0.5073991417884827 | 0.5073991417884827 |
| jensen_shannon_divergence | 1.676128625869751 | 1.676128625869751 | 1.676128625869751 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5092359185218811 | 0.5092359185218811 | 0.5092359185218811 |
| pearson_corr | 0.5886557698249817 | 0.5886557698249817 | 0.5886557698249817 |
| jensen_shannon_divergence | 1.4987353086471558 | 1.4987353086471558 | 1.4987353086471558 |


----------------------------



### 0.001, sigmoid, tanh

In [11]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')

    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.45377710461616516 | 0.45377710461616516 | 0.45377710461616516 |
| pearson_corr | 0.6438016891479492 | 0.6438016891479492 | 0.6438016891479492 |
| jensen_shannon_divergence | 1.1479465961456299 | 1.1479465961456299 | 1.1479465961456299 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.45613333582878113 | 0.45613333582878113 | 0.45613333582878113 |
| pearson_corr | 0.6516469120979309 | 0.6516469120979309 | 0.6516469120979309 |
| jensen_shannon_divergence | 1.204972505569458 | 1.204972505569458 | 1.204972505569458 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4835057854652405 | 0.4835057854652405 | 0.4835057854652405 |
| pearson_corr | 0.6221098899841309 | 0.6221098899841309 | 0.6221098899841309 |
| jensen_shannon_divergence | 1.3836482763290405 | 1.3836482763290405 | 1.3836482763290405 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4657197892665863 | 0.4657197892665863 | 0.4657197892665863 |
| pearson_corr | 0.6356847882270813 | 0.6356847882270813 | 0.6356847882270813 |
| jensen_shannon_divergence | 1.267802119255066 | 1.267802119255066 | 1.267802119255066 |


----------------------------



### 0.001, sigmoid, softmax

In [12]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')

    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.596490740776062 | 0.596490740776062 | 0.596490740776062 |
| pearson_corr | 0.5569028258323669 | 0.5569028258323669 | 0.5569028258323669 |
| jensen_shannon_divergence | 1.5869001150131226 | 1.5869001150131226 | 1.5869001150131226 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6342563033103943 | 0.6342563033103943 | 0.6342563033103943 |
| pearson_corr | 0.48862314224243164 | 0.48862314224243164 | 0.48862314224243164 |
| jensen_shannon_divergence | 1.8540529012680054 | 1.8540529012680054 | 1.8540529012680054 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5899628400802612 | 0.5899628400802612 | 0.5899628400802612 |
| pearson_corr | 0.4900575876235962 | 0.4900575876235962 | 0.4900575876235962 |
| jensen_shannon_divergence | 1.7186386585235596 | 1.7186386585235596 | 1.7186386585235596 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.49636921286582947 | 0.49636921286582947 | 0.49636921286582947 |
| pearson_corr | 0.6216035485267639 | 0.6216035485267639 | 0.6216035485267639 |
| jensen_shannon_divergence | 1.400059461593628 | 1.400059461593628 | 1.400059461593628 |


----------------------------



### 0.001, sigmoid, sigmoid

In [13]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')

    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5414367914199829 | 0.5414367914199829 | 0.5414367914199829 |
| pearson_corr | 0.5913442969322205 | 0.5913442969322205 | 0.5913442969322205 |
| jensen_shannon_divergence | 1.3468478918075562 | 1.3468478918075562 | 1.3468478918075562 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.45950567722320557 | 0.45950567722320557 | 0.45950567722320557 |
| pearson_corr | 0.6469155550003052 | 0.6469155550003052 | 0.6469155550003052 |
| jensen_shannon_divergence | 1.193283200263977 | 1.193283200263977 | 1.193283200263977 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.593237042427063 | 0.593237042427063 | 0.593237042427063 |
| pearson_corr | 0.5024652481079102 | 0.5024652481079102 | 0.5024652481079102 |
| jensen_shannon_divergence | 1.693637490272522 | 1.693637490272522 | 1.693637490272522 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4975337088108063 | 0.4975337088108063 | 0.4975337088108063 |
| pearson_corr | 0.6138879656791687 | 0.6138879656791687 | 0.6138879656791687 |
| jensen_shannon_divergence | 1.4269407987594604 | 1.4269407987594604 | 1.4269407987594604 |


----------------------------



### 0.001, sigmoid, relu

In [14]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5409899950027466 | 0.5409899950027466 | 0.5409899950027466 |
| pearson_corr | 0.5461926460266113 | 0.5461926460266113 | 0.5461926460266113 |
| jensen_shannon_divergence | 1.339079737663269 | 1.339079737663269 | 1.339079737663269 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4901297688484192 | 0.4901297688484192 | 0.4901297688484192 |
| pearson_corr | 0.6042608022689819 | 0.6042608022689819 | 0.6042608022689819 |
| jensen_shannon_divergence | 1.2799420356750488 | 1.2799420356750488 | 1.2799420356750488 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5829952955245972 | 0.5829952955245972 | 0.5829952955245972 |
| pearson_corr | 0.5115159749984741 | 0.5115159749984741 | 0.5115159749984741 |
| jensen_shannon_divergence | 1.700368046760559 | 1.700368046760559 | 1.700368046760559 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4951007664203644 | 0.4951007664203644 | 0.4951007664203644 |
| pearson_corr | 0.6182547211647034 | 0.6182547211647034 | 0.6182547211647034 |
| jensen_shannon_divergence | 1.4243485927581787 | 1.4243485927581787 | 1.4243485927581787 |


----------------------------



### 0.001, relu, tanh

In [15]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.450536847114563 | 0.450536847114563 | 0.450536847114563 |
| pearson_corr | 0.6518262624740601 | 0.6518262624740601 | 0.6518262624740601 |
| jensen_shannon_divergence | 1.1798590421676636 | 1.1798590421676636 | 1.1798590421676636 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.46526166796684265 | 0.46526166796684265 | 0.46526166796684265 |
| pearson_corr | 0.6264206767082214 | 0.6264206767082214 | 0.6264206767082214 |
| jensen_shannon_divergence | 1.2109544277191162 | 1.2109544277191162 | 1.2109544277191162 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4477587044239044 | 0.4477587044239044 | 0.4477587044239044 |
| pearson_corr | 0.6619058847427368 | 0.6619058847427368 | 0.6619058847427368 |
| jensen_shannon_divergence | 1.3054301738739014 | 1.3054301738739014 | 1.3054301738739014 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.475719690322876 | 0.475719690322876 | 0.475719690322876 |
| pearson_corr | 0.614625871181488 | 0.614625871181488 | 0.614625871181488 |
| jensen_shannon_divergence | 1.3954768180847168 | 1.3954768180847168 | 1.3954768180847168 |


----------------------------



### 0.001, relu, softmax

In [16]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5864970684051514 | 0.5864970684051514 | 0.5864970684051514 |
| pearson_corr | 0.5558068752288818 | 0.5558068752288818 | 0.5558068752288818 |
| jensen_shannon_divergence | 1.5315430164337158 | 1.5315430164337158 | 1.5315430164337158 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6390293836593628 | 0.6390293836593628 | 0.6390293836593628 |
| pearson_corr | 0.49406105279922485 | 0.49406105279922485 | 0.49406105279922485 |
| jensen_shannon_divergence | 1.8985143899917603 | 1.8985143899917603 | 1.8985143899917603 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5859909653663635 | 0.5859909653663635 | 0.5859909653663635 |
| pearson_corr | 0.48144587874412537 | 0.48144587874412537 | 0.48144587874412537 |
| jensen_shannon_divergence | 1.7363733053207397 | 1.7363733053207397 | 1.7363733053207397 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.49022528529167175 | 0.49022528529167175 | 0.49022528529167175 |
| pearson_corr | 0.6221278309822083 | 0.6221278309822083 | 0.6221278309822083 |
| jensen_shannon_divergence | 1.370262622833252 | 1.370262622833252 | 1.370262622833252 |


----------------------------



### 0.001, relu, sigmoid

In [17]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5307245254516602 | 0.5307245254516602 | 0.5307245254516602 |
| pearson_corr | 0.5653515458106995 | 0.5653515458106995 | 0.5653515458106995 |
| jensen_shannon_divergence | 1.331281304359436 | 1.331281304359436 | 1.331281304359436 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.46887487173080444 | 0.46887487173080444 | 0.46887487173080444 |
| pearson_corr | 0.6078193187713623 | 0.6078193187713623 | 0.6078193187713623 |
| jensen_shannon_divergence | 1.2894423007965088 | 1.2894423007965088 | 1.2894423007965088 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5768567323684692 | 0.5768567323684692 | 0.5768567323684692 |
| pearson_corr | 0.5316789150238037 | 0.5316789150238037 | 0.5316789150238037 |
| jensen_shannon_divergence | 1.6753581762313843 | 1.6753581762313843 | 1.6753581762313843 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4885828495025635 | 0.4885828495025635 | 0.4885828495025635 |
| pearson_corr | 0.6236727237701416 | 0.6236727237701416 | 0.6236727237701416 |
| jensen_shannon_divergence | 1.3382591009140015 | 1.3382591009140015 | 1.3382591009140015 |


----------------------------



### 0.001, relu, relu

In [18]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5424736738204956 | 0.5424736738204956 | 0.5424736738204956 |
| pearson_corr | 0.5311582088470459 | 0.5311582088470459 | 0.5311582088470459 |
| jensen_shannon_divergence | 1.3858336210250854 | 1.3858336210250854 | 1.3858336210250854 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47212597727775574 | 0.47212597727775574 | 0.47212597727775574 |
| pearson_corr | 0.6160202622413635 | 0.6160202622413635 | 0.6160202622413635 |
| jensen_shannon_divergence | 1.2930402755737305 | 1.2930402755737305 | 1.2930402755737305 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5976755023002625 | 0.5976755023002625 | 0.5976755023002625 |
| pearson_corr | 0.47857406735420227 | 0.47857406735420227 | 0.47857406735420227 |
| jensen_shannon_divergence | 1.8306392431259155 | 1.8306392431259155 | 1.8306392431259155 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47949063777923584 | 0.47949063777923584 | 0.47949063777923584 |
| pearson_corr | 0.6236826181411743 | 0.6236826181411743 | 0.6236826181411743 |
| jensen_shannon_divergence | 1.3180664777755737 | 1.3180664777755737 | 1.3180664777755737 |


----------------------------



---------------------------
### 0.01, tanh, tanh

In [19]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47185179591178894 | 0.47185179591178894 | 0.47185179591178894 |
| pearson_corr | 0.6154182553291321 | 0.6154182553291321 | 0.6154182553291321 |
| jensen_shannon_divergence | 1.217748761177063 | 1.217748761177063 | 1.217748761177063 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.48116570711135864 | 0.48116570711135864 | 0.48116570711135864 |
| pearson_corr | 0.612680971622467 | 0.612680971622467 | 0.612680971622467 |
| jensen_shannon_divergence | 1.2938724756240845 | 1.2938724756240845 | 1.2938724756240845 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5442043542861938 | 0.5442043542861938 | 0.5442043542861938 |
| pearson_corr | 0.5791927576065063 | 0.5791927576065063 | 0.5791927576065063 |
| jensen_shannon_divergence | 1.5329078435897827 | 1.5329078435897827 | 1.5329078435897827 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4966353476047516 | 0.4966353476047516 | 0.4966353476047516 |
| pearson_corr | 0.602621853351593 | 0.602621853351593 | 0.602621853351593 |
| jensen_shannon_divergence | 1.41573965549469 | 1.41573965549469 | 1.41573965549469 |


----------------------------



### 0.01, tanh, softmax

In [20]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5890794992446899 | 0.5890794992446899 | 0.5890794992446899 |
| pearson_corr | 0.5695134997367859 | 0.5695134997367859 | 0.5695134997367859 |
| jensen_shannon_divergence | 1.531375765800476 | 1.531375765800476 | 1.531375765800476 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6460238695144653 | 0.6460238695144653 | 0.6460238695144653 |
| pearson_corr | 0.49360039830207825 | 0.49360039830207825 | 0.49360039830207825 |
| jensen_shannon_divergence | 1.8824310302734375 | 1.8824310302734375 | 1.8824310302734375 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5912092328071594 | 0.5912092328071594 | 0.5912092328071594 |
| pearson_corr | 0.4810745120048523 | 0.4810745120048523 | 0.4810745120048523 |
| jensen_shannon_divergence | 1.727998971939087 | 1.727998971939087 | 1.727998971939087 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.49580878019332886 | 0.49580878019332886 | 0.49580878019332886 |
| pearson_corr | 0.614694356918335 | 0.614694356918335 | 0.614694356918335 |
| jensen_shannon_divergence | 1.4171581268310547 | 1.4171581268310547 | 1.4171581268310547 |


----------------------------



### 0.01, tanh, sigmoid

In [21]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5422801375389099 | 0.5422801375389099 | 0.5422801375389099 |
| pearson_corr | 0.587047278881073 | 0.587047278881073 | 0.587047278881073 |
| jensen_shannon_divergence | 1.3503721952438354 | 1.3503721952438354 | 1.3503721952438354 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47708481550216675 | 0.47708481550216675 | 0.47708481550216675 |
| pearson_corr | 0.6193339228630066 | 0.6193339228630066 | 0.6193339228630066 |
| jensen_shannon_divergence | 1.2904049158096313 | 1.2904049158096313 | 1.2904049158096313 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5933889150619507 | 0.5933889150619507 | 0.5933889150619507 |
| pearson_corr | 0.48848485946655273 | 0.48848485946655273 | 0.48848485946655273 |
| jensen_shannon_divergence | 1.7127653360366821 | 1.7127653360366821 | 1.7127653360366821 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.49638935923576355 | 0.49638935923576355 | 0.49638935923576355 |
| pearson_corr | 0.6168349385261536 | 0.6168349385261536 | 0.6168349385261536 |
| jensen_shannon_divergence | 1.421234130859375 | 1.421234130859375 | 1.421234130859375 |


----------------------------



### 0.01, tanh, relu

In [22]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5539107918739319 | 0.5539107918739319 | 0.5539107918739319 |
| pearson_corr | 0.539512038230896 | 0.539512038230896 | 0.539512038230896 |
| jensen_shannon_divergence | 1.3771109580993652 | 1.3771109580993652 | 1.3771109580993652 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4804069399833679 | 0.4804069399833679 | 0.4804069399833679 |
| pearson_corr | 0.6227656602859497 | 0.6227656602859497 | 0.6227656602859497 |
| jensen_shannon_divergence | 1.3032777309417725 | 1.3032777309417725 | 1.3032777309417725 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6472703218460083 | 0.6472703218460083 | 0.6472703218460083 |
| pearson_corr | 0.3543688952922821 | 0.3543688952922821 | 0.3543688952922821 |
| jensen_shannon_divergence | 2.0513923168182373 | 2.0513923168182373 | 2.0513923168182373 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4927695691585541 | 0.4927695691585541 | 0.4927695691585541 |
| pearson_corr | 0.6284472942352295 | 0.6284472942352295 | 0.6284472942352295 |
| jensen_shannon_divergence | 1.3787586688995361 | 1.3787586688995361 | 1.3787586688995361 |


----------------------------



### 0.01, softmax, tanh

In [23]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.463784396648407 | 0.463784396648407 | 0.463784396648407 |
| pearson_corr | 0.6310739517211914 | 0.6310739517211914 | 0.6310739517211914 |
| jensen_shannon_divergence | 1.1652400493621826 | 1.1652400493621826 | 1.1652400493621826 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4721786677837372 | 0.4721786677837372 | 0.4721786677837372 |
| pearson_corr | 0.6262760162353516 | 0.6262760162353516 | 0.6262760162353516 |
| jensen_shannon_divergence | 1.2640461921691895 | 1.2640461921691895 | 1.2640461921691895 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.537815272808075 | 0.537815272808075 | 0.537815272808075 |
| pearson_corr | 0.5780180096626282 | 0.5780180096626282 | 0.5780180096626282 |
| jensen_shannon_divergence | 1.5327320098876953 | 1.5327320098876953 | 1.5327320098876953 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.49045419692993164 | 0.49045419692993164 | 0.49045419692993164 |
| pearson_corr | 0.6118457913398743 | 0.6118457913398743 | 0.6118457913398743 |
| jensen_shannon_divergence | 1.372780442237854 | 1.372780442237854 | 1.372780442237854 |


----------------------------



### 0.01, softmax, softmax

In [24]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5884732604026794 | 0.5884732604026794 | 0.5884732604026794 |
| pearson_corr | 0.5692443251609802 | 0.5692443251609802 | 0.5692443251609802 |
| jensen_shannon_divergence | 1.527906894683838 | 1.527906894683838 | 1.527906894683838 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6394171118736267 | 0.6394171118736267 | 0.6394171118736267 |
| pearson_corr | 0.5088158845901489 | 0.5088158845901489 | 0.5088158845901489 |
| jensen_shannon_divergence | 1.8363369703292847 | 1.8363369703292847 | 1.8363369703292847 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5911292433738708 | 0.5911292433738708 | 0.5911292433738708 |
| pearson_corr | 0.4821214973926544 | 0.4821214973926544 | 0.4821214973926544 |
| jensen_shannon_divergence | 1.724567174911499 | 1.724567174911499 | 1.724567174911499 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.49522846937179565 | 0.49522846937179565 | 0.49522846937179565 |
| pearson_corr | 0.6152331233024597 | 0.6152331233024597 | 0.6152331233024597 |
| jensen_shannon_divergence | 1.4066143035888672 | 1.4066143035888672 | 1.4066143035888672 |


----------------------------



### 0.01, softmax, sigmoid

In [25]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5413151383399963 | 0.5413151383399963 | 0.5413151383399963 |
| pearson_corr | 0.5919100642204285 | 0.5919100642204285 | 0.5919100642204285 |
| jensen_shannon_divergence | 1.3468931913375854 | 1.3468931913375854 | 1.3468931913375854 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47169631719589233 | 0.47169631719589233 | 0.47169631719589233 |
| pearson_corr | 0.6243536472320557 | 0.6243536472320557 | 0.6243536472320557 |
| jensen_shannon_divergence | 1.2730300426483154 | 1.2730300426483154 | 1.2730300426483154 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5935360789299011 | 0.5935360789299011 | 0.5935360789299011 |
| pearson_corr | 0.5019571781158447 | 0.5019571781158447 | 0.5019571781158447 |
| jensen_shannon_divergence | 1.6946351528167725 | 1.6946351528167725 | 1.6946351528167725 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.49768906831741333 | 0.49768906831741333 | 0.49768906831741333 |
| pearson_corr | 0.6135697960853577 | 0.6135697960853577 | 0.6135697960853577 |
| jensen_shannon_divergence | 1.427812933921814 | 1.427812933921814 | 1.427812933921814 |


----------------------------



### 0.01, softmax, relu

In [26]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5456269383430481 | 0.5456269383430481 | 0.5456269383430481 |
| pearson_corr | 0.5812503695487976 | 0.5812503695487976 | 0.5812503695487976 |
| jensen_shannon_divergence | 1.3451234102249146 | 1.3451234102249146 | 1.3451234102249146 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.46669766306877136 | 0.46669766306877136 | 0.46669766306877136 |
| pearson_corr | 0.6330732703208923 | 0.6330732703208923 | 0.6330732703208923 |
| jensen_shannon_divergence | 1.2417933940887451 | 1.2417933940887451 | 1.2417933940887451 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5886719226837158 | 0.5886719226837158 | 0.5886719226837158 |
| pearson_corr | 0.5015778541564941 | 0.5015778541564941 | 0.5015778541564941 |
| jensen_shannon_divergence | 1.6870161294937134 | 1.6870161294937134 | 1.6870161294937134 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5244675278663635 | 0.5244675278663635 | 0.5244675278663635 |
| pearson_corr | 0.5695959329605103 | 0.5695959329605103 | 0.5695959329605103 |
| jensen_shannon_divergence | 1.5786405801773071 | 1.5786405801773071 | 1.5786405801773071 |


----------------------------



### 0.01, sigmoid, tanh

In [27]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.46929019689559937 | 0.46929019689559937 | 0.46929019689559937 |
| pearson_corr | 0.6296336054801941 | 0.6296336054801941 | 0.6296336054801941 |
| jensen_shannon_divergence | 1.2746973037719727 | 1.2746973037719727 | 1.2746973037719727 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4660244286060333 | 0.4660244286060333 | 0.4660244286060333 |
| pearson_corr | 0.635574221611023 | 0.635574221611023 | 0.635574221611023 |
| jensen_shannon_divergence | 1.2561781406402588 | 1.2561781406402588 | 1.2561781406402588 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47374892234802246 | 0.47374892234802246 | 0.47374892234802246 |
| pearson_corr | 0.6404688358306885 | 0.6404688358306885 | 0.6404688358306885 |
| jensen_shannon_divergence | 1.3556456565856934 | 1.3556456565856934 | 1.3556456565856934 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4523907005786896 | 0.4523907005786896 | 0.4523907005786896 |
| pearson_corr | 0.6476398706436157 | 0.6476398706436157 | 0.6476398706436157 |
| jensen_shannon_divergence | 1.2991381883621216 | 1.2991381883621216 | 1.2991381883621216 |


----------------------------



### 0.01, sigmoid, softmax

In [28]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5928510427474976 | 0.5928510427474976 | 0.5928510427474976 |
| pearson_corr | 0.5661187767982483 | 0.5661187767982483 | 0.5661187767982483 |
| jensen_shannon_divergence | 1.5550379753112793 | 1.5550379753112793 | 1.5550379753112793 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6393083333969116 | 0.6393083333969116 | 0.6393083333969116 |
| pearson_corr | 0.5068081617355347 | 0.5068081617355347 | 0.5068081617355347 |
| jensen_shannon_divergence | 1.8494629859924316 | 1.8494629859924316 | 1.8494629859924316 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5923035740852356 | 0.5923035740852356 | 0.5923035740852356 |
| pearson_corr | 0.48401114344596863 | 0.48401114344596863 | 0.48401114344596863 |
| jensen_shannon_divergence | 1.7242666482925415 | 1.7242666482925415 | 1.7242666482925415 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5001022815704346 | 0.5001022815704346 | 0.5001022815704346 |
| pearson_corr | 0.6033743619918823 | 0.6033743619918823 | 0.6033743619918823 |
| jensen_shannon_divergence | 1.4290117025375366 | 1.4290117025375366 | 1.4290117025375366 |


----------------------------



### 0.01, sigmoid, sigmoid

In [29]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5428560376167297 | 0.5428560376167297 | 0.5428560376167297 |
| pearson_corr | 0.5865004658699036 | 0.5865004658699036 | 0.5865004658699036 |
| jensen_shannon_divergence | 1.352591633796692 | 1.352591633796692 | 1.352591633796692 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.48195162415504456 | 0.48195162415504456 | 0.48195162415504456 |
| pearson_corr | 0.6206144094467163 | 0.6206144094467163 | 0.6206144094467163 |
| jensen_shannon_divergence | 1.2290713787078857 | 1.2290713787078857 | 1.2290713787078857 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5933076739311218 | 0.5933076739311218 | 0.5933076739311218 |
| pearson_corr | 0.49069565534591675 | 0.49069565534591675 | 0.49069565534591675 |
| jensen_shannon_divergence | 1.708763599395752 | 1.708763599395752 | 1.708763599395752 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.496796190738678 | 0.496796190738678 | 0.496796190738678 |
| pearson_corr | 0.6139892935752869 | 0.6139892935752869 | 0.6139892935752869 |
| jensen_shannon_divergence | 1.415961742401123 | 1.415961742401123 | 1.415961742401123 |


----------------------------



### 0.01, sigmoid, relu

In [30]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5543439984321594 | 0.5543439984321594 | 0.5543439984321594 |
| pearson_corr | 0.5205754637718201 | 0.5205754637718201 | 0.5205754637718201 |
| jensen_shannon_divergence | 1.3660242557525635 | 1.3660242557525635 | 1.3660242557525635 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4828846752643585 | 0.4828846752643585 | 0.4828846752643585 |
| pearson_corr | 0.603183925151825 | 0.603183925151825 | 0.603183925151825 |
| jensen_shannon_divergence | 1.4078378677368164 | 1.4078378677368164 | 1.4078378677368164 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5830832719802856 | 0.5830832719802856 | 0.5830832719802856 |
| pearson_corr | 0.48991090059280396 | 0.48991090059280396 | 0.48991090059280396 |
| jensen_shannon_divergence | 1.761276125907898 | 1.761276125907898 | 1.761276125907898 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5000730156898499 | 0.5000730156898499 | 0.5000730156898499 |
| pearson_corr | 0.6181396842002869 | 0.6181396842002869 | 0.6181396842002869 |
| jensen_shannon_divergence | 1.394651174545288 | 1.394651174545288 | 1.394651174545288 |


----------------------------



### 0.01, relu, tanh

In [31]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4580466151237488 | 0.4580466151237488 | 0.4580466151237488 |
| pearson_corr | 0.6450393199920654 | 0.6450393199920654 | 0.6450393199920654 |
| jensen_shannon_divergence | 1.1763347387313843 | 1.1763347387313843 | 1.1763347387313843 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4524895250797272 | 0.4524895250797272 | 0.4524895250797272 |
| pearson_corr | 0.6494733691215515 | 0.6494733691215515 | 0.6494733691215515 |
| jensen_shannon_divergence | 1.2070879936218262 | 1.2070879936218262 | 1.2070879936218262 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5588112473487854 | 0.5588112473487854 | 0.5588112473487854 |
| pearson_corr | 0.5562357902526855 | 0.5562357902526855 | 0.5562357902526855 |
| jensen_shannon_divergence | 1.6098650693893433 | 1.6098650693893433 | 1.6098650693893433 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5008090138435364 | 0.5008090138435364 | 0.5008090138435364 |
| pearson_corr | 0.5950855016708374 | 0.5950855016708374 | 0.5950855016708374 |
| jensen_shannon_divergence | 1.433365821838379 | 1.433365821838379 | 1.433365821838379 |


----------------------------



### 0.01, relu, softmax

In [32]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7453762888908386 | 0.7453762888908386 | 0.7453762888908386 |
| pearson_corr | 0.24130350351333618 | 0.24130350351333618 | 0.24130350351333618 |
| jensen_shannon_divergence | 2.6138551235198975 | 2.6138551235198975 | 2.6138551235198975 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6372242569923401 | 0.6372242569923401 | 0.6372242569923401 |
| pearson_corr | 0.5124882459640503 | 0.5124882459640503 | 0.5124882459640503 |
| jensen_shannon_divergence | 1.8456066846847534 | 1.8456066846847534 | 1.8456066846847534 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5945754051208496 | 0.5945754051208496 | 0.5945754051208496 |
| pearson_corr | 0.4792262017726898 | 0.4792262017726898 | 0.4792262017726898 |
| jensen_shannon_divergence | 1.741890549659729 | 1.741890549659729 | 1.741890549659729 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.48866403102874756 | 0.48866403102874756 | 0.48866403102874756 |
| pearson_corr | 0.6273646950721741 | 0.6273646950721741 | 0.6273646950721741 |
| jensen_shannon_divergence | 1.340421438217163 | 1.340421438217163 | 1.340421438217163 |


----------------------------



### 0.01, relu, sigmoid

In [33]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5712888836860657 | 0.5712888836860657 | 0.5712888836860657 |
| pearson_corr | 0.44382306933403015 | 0.44382306933403015 | 0.44382306933403015 |
| jensen_shannon_divergence | 1.4426497220993042 | 1.4426497220993042 | 1.4426497220993042 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4597261846065521 | 0.4597261846065521 | 0.4597261846065521 |
| pearson_corr | 0.6482595205307007 | 0.6482595205307007 | 0.6482595205307007 |
| jensen_shannon_divergence | 1.2516006231307983 | 1.2516006231307983 | 1.2516006231307983 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5898919105529785 | 0.5898919105529785 | 0.5898919105529785 |
| pearson_corr | 0.5116769671440125 | 0.5116769671440125 | 0.5116769671440125 |
| jensen_shannon_divergence | 1.7275757789611816 | 1.7275757789611816 | 1.7275757789611816 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.485655277967453 | 0.485655277967453 | 0.485655277967453 |
| pearson_corr | 0.611379861831665 | 0.611379861831665 | 0.611379861831665 |
| jensen_shannon_divergence | 1.3229633569717407 | 1.3229633569717407 | 1.3229633569717407 |


----------------------------



### 0.01, relu, relu

In [34]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5754036903381348 | 0.5754036903381348 | 0.5754036903381348 |
| pearson_corr | 0.5168760418891907 | 0.5168760418891907 | 0.5168760418891907 |
| jensen_shannon_divergence | 1.4924464225769043 | 1.4924464225769043 | 1.4924464225769043 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4638281464576721 | 0.4638281464576721 | 0.4638281464576721 |
| pearson_corr | 0.6322964429855347 | 0.6322964429855347 | 0.6322964429855347 |
| jensen_shannon_divergence | 1.3287949562072754 | 1.3287949562072754 | 1.3287949562072754 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6472703218460083 | 0.6472703218460083 | 0.6472703218460083 |
| pearson_corr | 0.3543688952922821 | 0.3543688952922821 | 0.3543688952922821 |
| jensen_shannon_divergence | 2.0513923168182373 | 2.0513923168182373 | 2.0513923168182373 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5292101502418518 | 0.5292101502418518 | 0.5292101502418518 |
| pearson_corr | 0.5924198627471924 | 0.5924198627471924 | 0.5924198627471924 |
| jensen_shannon_divergence | 1.5824764966964722 | 1.5824764966964722 | 1.5824764966964722 |


----------------------------

